# <font color='blue'>Rough Bergomi Tree</font>

1. Understand Jim's notebook on how to calibrate $(H, \rho, \nu)$ based on market smile data.
2. Given a set of smiles, possibly at differnt expiries $\tau$, estimate $(H, \rho, \nu)$ .
3. ~~Implement the data structure of bushy rBergomi tree.~~
4. Implement forward evolution of stock:
    - ~~forward evolution of $B_t$;~~
    - ~~forward evolution of $W_t$ and $\varphi(t)=\mathcal{G}^{\alpha}(W_t)$, where $\alpha = H-1/2$, or $2\alpha+1=2H$;~~
    - ~~valuation of $\Phi(\varphi)(t) = \xi_0(t) \exp[2\nu C_H \varphi(t)-2\nu^2 C_H^2 \int_0^t (t-s)^{2\alpha}ds] = \xi_0(t) \exp[2\nu C_H \varphi(t)-\nu^2 C_H^2 t^{2H}/H]$. Initially, $\Phi(\varphi)(t=0)=\xi_0(0)$;~~
    - ~~forward evolution of $X_t$.~~
5. Implement backward valuation of a European option:
    - ~~Set terminal payoff;~~
    - ~~Given $(H, \rho, \nu, S_0, K, r, q, \xi_0(\tau))$, return an option price;~~
    - ~~Given $(\rho, \nu, S_0, K, r, q, \xi_0(\tau))$ and a range of $H$'s, return a list of option prices;~~
    - ~~Use Black-Scholes formula to generate the vanilla smile.~~
6. Implement backward valuation of an American option:
    - ~~Given $(H, \rho, \nu, S_0, K, r, q, \xi_0(\tau))$, return an option price;~~
    - ~~Given $(\rho, \nu, S_0, K, r, q, \xi_0(\tau))$ and a range of $H$'s, return a list of option prices;~~
7. ~~Take difference between the European option prices and the American optoin prices to get the early exercise premium, over a range of moneyness.~~
8. Loop over a variety of Hurst $H$'s with $(\rho, \nu)$ calibrated to the given market smile data.

In [2]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

import math
import numpy as np
import pandas as pd
from structure import *
from black_model import black_impv
from rough_bergomi_model import RoughBergomiModel
from local_volatility_model import LocalVolatilityModel
from run_tests import *

## <font color='gray'>Parameters</font>

In [3]:
# User-input parameters:

# Time horizon 
T = 1.0
# Number of time intervals
N = 10
# Hurst exponent
H = 0.05
# Correlation
rho = -0.9
# Coefficient of fractional BM
nu = 1.0
# Current value of variance
xi0 = 0.04
# Current value of stock
S0 = 1.0
# Risk-free interest rate
r = 0.05
# Method to evolve stocks
EVOLVE_METHOD='dLogS'
# Forward variance curve
def fvc(t): return xi0

## <font color='gray'>Run Tests: Volatility of Volatility</font>

In [33]:
nuList = np.linspace(0.0, 2.0, num=41)[1:]
nuList

array([ 0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,  0.4 ,  0.45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ,  1.05,  1.1 ,  1.15,  1.2 ,  1.25,  1.3 ,  1.35,
        1.4 ,  1.45,  1.5 ,  1.55,  1.6 ,  1.65,  1.7 ,  1.75,  1.8 ,
        1.85,  1.9 ,  1.95,  2.  ])

In [34]:
res = run_vol_of_vol(T, nuList, rho, H, fvc, r, S0, N)

Klist_rBergomi_vv = res[0]
euput_rBergomi_results_vv = res[1]
amput_rBergomi_results_vv = res[2]
Klist_localvol_vv = res[3]
euput_localvol_results_vv = res[4]
amput_localvol_results_vv = res[5]

Hurst parameter H: 0.05
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 0.05
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.023871
ν^2*C_H^2/H: 0.002849
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.41 2.225
LogX span: -0.891640661822 0.799642661872
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.758 1.294
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.675 1.438
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Ro

Rough Bergomi - Tree span at step 10 : 0.743 1.265
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.656 1.39
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.595 1.494
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.548 1.587
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.509 1.673
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.477 1.755
Rough Bergomi - Number of time steps: 10
Rough Bergomi

Rough Bergomi - Tree span at step 10 : 0.489 1.58
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.456 1.647
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.427 1.711
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.402 1.773
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.38 1.832
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.36 1.89
Mannual override calls (A.1): 55 56 45 46
****************

Rough Bergomi - Tree span at step 10 : 0.366 1.694
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.346 1.74
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 0.55
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.262576
ν^2*C_H^2/H: 0.34473
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.343 1.694
LogX span: -1.07023663787 0.527206963472
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree s

Rough Bergomi - Tree span at step 10 : 0.716 1.169
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.622 1.24
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.558 1.296
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.509 1.345
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.469 1.388
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.435 1.429
Rough Bergomi - Number of time steps: 10
Rough Bergomi

...

Rough Bergomi - Tree span at step 10 : 0.663 1.118
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.608 1.142
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.565 1.162
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.53 1.18
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.501 1.196
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.476 1.211
Mannual override calls (B.1): 28 29 21 2

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.561 1.155
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.534 1.168
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.51 1.181
Mannual override calls (B.1): 28 29 21 22
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.489 1.192
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.47 1.203
Mannual override calls (B.1): 46 47 37 38
Mannual override calls 

Rough Bergomi - Tree span at step 10 : 0.549 1.156
Mannual override calls (B.1): 28 29 21 22
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.53 1.166
Mannual override calls (A.2): 36 1.165644254
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.512 1.175
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.496 1.183
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.7
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.811598
ν^2*C_H^2/H: 3.293453
Rough Bergomi - Number of time steps: 10
Rough 

Mannual override calls (B.1): 46 47 37 38
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.544 1.16
Mannual override calls (A.1): 56 57 46 47
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.85
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.883209
ν^2*C_H^2/H: 3.900291
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.561 1.153
LogX span: -0.578388461822 0.142488294854
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time 

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.595 1.141
Mannual override calls (A.1): 58 59 48 49
Mannual override calls (A.1): 57 58 47 48
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 2.0
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.954821
ν^2*C_H^2/H: 4.558412
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.612 1.136
LogX span: -0.491573570886 0.127611963786
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time 

In [36]:
import pickle

In [52]:
# with open('Klist_localvol', 'wb') as fp:
#     pickle.dump(Klist_localvol, fp)
with open ('euput_rBergomi_results_vv', 'rb') as fp:
    euput_rBergomi_results_vv = pickle.load(fp)
with open ('amput_rBergomi_results_vv', 'rb') as fp:
    amput_rBergomi_results_vv = pickle.load(fp)
with open ('euput_localvol_results_vv', 'rb') as fp:
    euput_localvol_results_vv = pickle.load(fp)
with open ('amput_localvol_results_vv', 'rb') as fp:
    amput_localvol_results_vv = pickle.load(fp)
with open ('Klist_rBergomi_vv', 'rb') as fp:
    Klist_rBergomi_vv = pickle.load(fp)
with open ('Klist_localvol_vv', 'rb') as fp:
    Klist_localvol_vv = pickle.load(fp)

In [54]:
euput_rBergomi_results_vv = np.array(euput_rBergomi_results_vv)
amput_rBergomi_results_vv = np.array(amput_rBergomi_results_vv)
euput_localvol_results_vv = np.array(euput_localvol_results_vv)
amput_localvol_results_vv = np.array(amput_localvol_results_vv)

In [58]:
amput_localvol_results_vv.shape

(40, 31)

In [44]:
# import matplotlib.pyplot as plt
# plt.plot(np.log(Klist_rBergomi), list(eep_rBergomi - eep_localvol)[0])

## <font color='gray'>Run Tests: Correlation</font>

In [63]:
rhoList = np.linspace(-1.0, 1.0, num=41)[1:-1]
rhoList

array([-0.95, -0.9 , -0.85, -0.8 , -0.75, -0.7 , -0.65, -0.6 , -0.55,
       -0.5 , -0.45, -0.4 , -0.35, -0.3 , -0.25, -0.2 , -0.15, -0.1 ,
       -0.05,  0.  ,  0.05,  0.1 ,  0.15,  0.2 ,  0.25,  0.3 ,  0.35,
        0.4 ,  0.45,  0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,
        0.85,  0.9 ,  0.95])

In [64]:
res = run_correlation(T, rhoList, nu, H, fvc, r, S0, N)

Hurst parameter H: 0.05
Correlation coefficient 1: -0.95
Correlation coefficient 2: 0.312
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.47741
ν^2*C_H^2/H: 1.139603
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.376 1.351
LogX span: -0.977202325462 0.30110541159
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.733 1.115
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.645 1.16
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Roug

Rough Bergomi - Tree span at step 10 : 0.709 1.131
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.615 1.184
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.552 1.225
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.504 1.26
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.465 1.291
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.433 1.32
Rough Bergomi - Number of time steps: 10
Rough Bergomi 

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.43 1.353
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.402 1.383
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.378 1.411
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.356 1.438
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.337 1.464
*********************************************************************
Hurst parameter H: 0.05
Correlation 

Rough Bergomi - Tree span at step 10 : 0.348 1.534
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: -0.45
Correlation coefficient 2: 0.893
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.47741
ν^2*C_H^2/H: 1.139603
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.354 1.574
LogX span: -1.0378431046 0.453343796806
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.719 1.168
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree s

Rough Bergomi - Tree span at step 10 : 0.734 1.199
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.647 1.288
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.587 1.36
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.541 1.422
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.503 1.479
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.471 1.531
Rough Bergomi - Number of time steps: 10
Rough Bergomi

...

Rough Bergomi - Tree span at step 10 : 0.699 1.622
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.662 1.742
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.63 1.853
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.602 1.96
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.578 2.062
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.556 2.162
Rough Bergomi - Number of time steps: 10
Rough Bergomi -

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.606 2.323
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.587 2.438
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.571 2.551
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: 0.4
Correlation coefficient 2: 0.917
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.47741
ν^2*C_H^2/H: 1.139603
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.59 2.608
LogX span: -0.

Rough Bergomi - Tree span at step 10 : 0.644 2.746
LogX span: -0.439829583992 1.0102197377
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.866 1.397
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.819 1.597
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.784 1.767
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.755 1.922
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes:

Rough Bergomi - Tree span at step 10 : 0.802 1.788
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.778 1.949
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.756 2.101
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.738 2.248
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.721 2.391
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.706 2.531
Rough Bergomi - Number of time steps: 10
Rough Bergomi

Rough Bergomi - Tree span at step 10 : 0.745 2.335
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.731 2.468
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.719 2.599
Mannual override puts (B.1): 54 53 44 43
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.707 2.729
*********************************************************************
Hurst parameter H: 0.05
Correlation coefficient 1: 0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.45
2H: 0.1
C_H: 0.238705
2*ν*C_H: 0.47741
ν^2*C_H^2/H: 1.139603
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough 

In [65]:
Klist_rBergomi_rho = res[0]
euput_rBergomi_results_rho = res[1]
amput_rBergomi_results_rho = res[2]
Klist_localvol_rho = res[3]
euput_localvol_results_rho = res[4]
amput_localvol_results_rho = res[5]

In [71]:
# with open('amput_localvol_results_rho', 'wb') as fp:
#     pickle.dump(amput_localvol_results_rho, fp)

## <font color='gray'>Run Tests: Roughness</font>

In [79]:
HList = np.linspace(0.0, 0.5, num=51)[1:]
HList

array([ 0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,
        0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17,  0.18,
        0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,  0.26,  0.27,
        0.28,  0.29,  0.3 ,  0.31,  0.32,  0.33,  0.34,  0.35,  0.36,
        0.37,  0.38,  0.39,  0.4 ,  0.41,  0.42,  0.43,  0.44,  0.45,
        0.46,  0.47,  0.48,  0.49,  0.5 ])

In [80]:
res = run_roughness(T, HList, nu, rho, fvc, r, S0, N)

Klist_rBergomi_H = res[0]
euput_rBergomi_results_H = res[1]
amput_rBergomi_results_H = res[2]
Klist_localvol_H = res[3]
euput_localvol_results_H = res[4]
amput_localvol_results_H = res[5]

Hurst parameter H: 0.01
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.49
2H: 0.02
C_H: 0.101379
2*ν*C_H: 0.202759
ν^2*C_H^2/H: 1.027778
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.483 1.52
LogX span: -0.7272244172 0.4187887393
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.795 1.147
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.723 1.211
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough 

Rough Bergomi - Tree span at step 10 : 0.735 1.125
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.648 1.176
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.588 1.215
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.542 1.248
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.505 1.278
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.473 1.305
Rough Bergomi - Number of time steps: 10
Rough Bergom

Rough Bergomi - Tree span at step 10 : 0.441 1.254
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.407 1.276
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.379 1.297
Mannual override calls (B.2): 28 1.29748677159
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.354 1.317
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.332 1.336
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.313 1.354
*******

Rough Bergomi - Tree span at step 10 : 0.279 1.321
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.26 1.337
*********************************************************************
Hurst parameter H: 0.11
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.39
2H: 0.22
C_H: 0.379188
2*ν*C_H: 0.758376
ν^2*C_H^2/H: 1.307121
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.245 1.332
LogX span: -1.40591522187 0.286925436056
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree 

Rough Bergomi - Tree span at step 10 : 0.636 1.123
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.516 1.165
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.439 1.196
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.382 1.221
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.337 1.242
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.301 1.262
Rough Bergomi - Number of time steps: 10
Rough Bergom

...

Rough Bergomi - Tree span at step 10 : 0.617 1.15
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.457 1.19
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.35 1.22
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.273 1.243
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.216 1.262
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.173 1.279
Rough Bergomi - Number of time steps: 10
Rough Bergomi - 

Rough Bergomi - Tree span at step 10 : 0.166 1.289
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.132 1.304
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.105 1.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.085 1.326
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.068 1.334
*********************************************************************
Hurst parameter H: 0.38
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.12
2H: 0.7

Rough Bergomi - Tree span at step 10 : 0.061 1.351
LogX span: -2.8030589708 0.300850482327
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.633 1.165
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.468 1.206
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.353 1.237
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.27 1.262
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 

Rough Bergomi - Tree span at step 10 : 0.358 1.245
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.272 1.272
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.207 1.296
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.159 1.317
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.122 1.334
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.094 1.347
Rough Bergomi - Number of time steps: 10
Rough Bergomi

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.092 1.364
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.07 1.378
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.053 1.389
*********************************************************************
Hurst parameter H: 0.48
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.0
Forward variance curve (flat): 0.04
α:  -0.02
2H: 0.96
C_H: 0.979161
2*ν*C_H: 1.958321
ν^2*C_H^2/H: 1.997407
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.053 1.397
LogX span: 

In [87]:
# with open('amput_localvol_results_H', 'wb') as fp:
#     pickle.dump(amput_localvol_results_H, fp)

## <font color='gray'>Run Tests: Roughness with $\eta\triangleq 2\nu C_H$ fixed</font>

In [11]:
HList = np.linspace(0.0, 0.5, num=51)[3:]
HList

array([ 0.03,  0.04,  0.05,  0.06,  0.07,  0.08,  0.09,  0.1 ,  0.11,
        0.12,  0.13,  0.14,  0.15,  0.16,  0.17,  0.18,  0.19,  0.2 ,
        0.21,  0.22,  0.23,  0.24,  0.25,  0.26,  0.27,  0.28,  0.29,
        0.3 ,  0.31,  0.32,  0.33,  0.34,  0.35,  0.36,  0.37,  0.38,
        0.39,  0.4 ,  0.41,  0.42,  0.43,  0.44,  0.45,  0.46,  0.47,
        0.48,  0.49,  0.5 ])

In [12]:
eta = 0.7
res = run_roughness_fixed_eta(T, HList, eta, rho, fvc, r, S0, N)

Klist_rBergomi_H_fixed_eta = res[0]
euput_rBergomi_results_H_fixed_eta = res[1]
amput_rBergomi_results_H_fixed_eta = res[2]
Klist_localvol_H_fixed_eta = res[3]
euput_localvol_results_H_fixed_eta = res[4]
amput_localvol_results_H_fixed_eta = res[5]

Hurst parameter H: 0.03
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 1.9412219773059525
Forward variance curve (flat): 0.04
α:  -0.47
2H: 0.06
C_H: 0.180299
2*ν*C_H: 0.7
ν^2*C_H^2/H: 4.083333
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.661 1.144
LogX span: -0.413833033934 0.13459134095
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.861 1.049
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.816 1.068
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 

Rough Bergomi - Tree span at step 10 : 0.38 1.252
LogX span: -0.967118760098 0.224994545553
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.723 1.089
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.637 1.121
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.578 1.145
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.534 1.166
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes

Rough Bergomi - Tree span at step 10 : 0.568 1.157
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.499 1.189
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.448 1.215
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.406 1.238
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.372 1.258
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.344 1.277
Rough Bergomi - Number of time steps: 

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.303 1.321
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.277 1.341
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.255 1.36
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.235 1.377
*********************************************************************
Hurst parameter H: 0.13
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 0.8317301701747527
Forward variance curve (flat): 0.04
α:  -0.37
2H: 0.26
C_H: 0.42081
2*ν*C_H: 0.7
ν^2*C_H^2

Rough Bergomi - Tree span at step 10 : 0.212 1.426
LogX span: -1.55256391821 0.354646552634
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.657 1.157
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.538 1.21
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.459 1.251
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.399 1.284
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes:

...

Rough Bergomi - Tree span at step 10 : 0.216 1.578
LogX span: -1.53040851485 0.456458300482
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.707 1.227
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.589 1.307
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.505 1.364
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.44 1.409
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes:

Rough Bergomi - Tree span at step 10 : 0.514 1.377
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.448 1.424
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.394 1.463
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.349 1.497
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.31 1.526
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.276 1.553
Rough Bergomi - Number of time steps: 10
Rough Bergomi 

Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.283 1.57
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.9
Rough Bergomi - Tree span at step 10 : 0.253 1.595
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.226 1.617
*********************************************************************
Hurst parameter H: 0.43
Correlation coefficient 1: -0.9
Correlation coefficient 2: 0.436
Volatility of volatility: 0.3803301865766085
Forward variance curve (flat): 0.04
α:  -0.07
2H: 0.86
C_H: 0.920253
2*ν*C_H: 0.7
ν^2*C_H^2/H: 0.284884
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 1.0
Rough Bergomi - Tree span at step 10 : 0.228 1.623
L

Rough Bergomi - Tree span at step 10 : 0.233 1.64
LogX span: -1.45504224747 0.494782916017
Number of time steps: 10
Total number of nodes: 66
Time horizon: 1.0
Time step width: 0.1
Square root of time step: 0.316
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.1
Rough Bergomi - Tree span at step 10 : 0.724 1.248
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.2
Rough Bergomi - Tree span at step 10 : 0.613 1.339
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.30000000000000004
Rough Bergomi - Tree span at step 10 : 0.531 1.404
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.466 1.455
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes:

Rough Bergomi - Tree span at step 10 : 0.538 1.414
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.4
Rough Bergomi - Tree span at step 10 : 0.473 1.467
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.5
Rough Bergomi - Tree span at step 10 : 0.419 1.511
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.6000000000000001
Rough Bergomi - Tree span at step 10 : 0.372 1.548
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.7000000000000001
Rough Bergomi - Tree span at step 10 : 0.332 1.58
Rough Bergomi - Number of time steps: 10
Rough Bergomi - Total number of nodes: 1398101
Rough Bergomi - Time horizon: 0.8
Rough Bergomi - Tree span at step 10 : 0.297 1.609
Rough Bergomi - Number of time steps: 10
Rough Bergomi 

In [19]:
import pickle
with open('amput_localvol_results_H_fixed_eta', 'wb') as fp:
     pickle.dump(amput_localvol_results_H_fixed_eta, fp)

## <font color='gray'>Rough Bergomi Model</font>

In [ ]:
rBergomi = RoughBergomiModel(H, rho, nu, fvc, r)

In [ ]:
#rBergomi.draw_fbm_tree(N, T)

In [ ]:
Xmin, Xmax = rBergomi.create_price_tree(N, T, S0, EVOLVE_METHOD="dLogS", DEBUG=False)
print("LogX span:", np.log(Xmin), np.log(Xmax))

In [ ]:
rBergomi.draw_price_tree(N, T)

In [ ]:
num_moneyness = 60
moneyness_pixel = (np.log(Xmax) - np.log(Xmin))/60
LogMoneyness_list = np.linspace(-0.3, 0.29, num=num_moneyness)
Klist = S0*np.exp(LogMoneyness_list)

# <font color='blue'>Local Volatility Model Calibrated to Rough Bergomi Smile</font>

In [ ]:
localVol = LocalVolatilityModel(r=0.0)
localVolN = N
localVol.create_tree(localVolN, T, S0, rBergomi, N, False)

In [ ]:
localVol.draw_price_tree(localVolN,T)

In [ ]:
euput_rBergomi,_ = rBergomi.price_option(N, T, Klist, opttype=-1, earlyexercise=False, r=0.0)

In [ ]:
LogMoneyness_list = np.linspace(-0.3, 0.29, num=30)
Klist_short = S0*np.exp(LogMoneyness_list)
euput_local_vol,_ = localVol.price_option(N, T, Klist_short, opttype=-1, earlyexercise=False, r=0.0)

In [ ]:
start_index = 0

df1 = pd.DataFrame(index=np.log(Klist)[start_index:])
df1["European Put (Rough Bergomi Model)"] = euput_rBergomi[start_index:]

df2 = pd.DataFrame(index=np.log(Klist_short)[start_index:])
df2["European Put (Local Volatility Model)"] = euput_local_vol[start_index:]

fig, axes = plt.subplots()
axes.plot(df1.index, df1["European Put (Rough Bergomi Model)"],'b-')
axes.plot(df2.index, df2["European Put (Local Volatility Model)"], 'ro', mfc='none', markersize=12)       
plt.legend(prop={'size':30})
plt.xlabel('Log-Moneyness', fontsize=30)
plt.ylabel("European Put Options (Zero Interest Rate)", fontsize=30)
plt.rcParams["figure.figsize"] = (20,15)

In [ ]:
euput_rBergomi_with_interest,_ = rBergomi.price_option(N, T, Klist, opttype=-1, earlyexercise=False, r=0.05)
amput_rBergomi_with_interest,boundaries = rBergomi.price_option(N, T, Klist, opttype=-1, earlyexercise=True, r=0.05)

# <font color='blue'>Early Exercise Boundaries</font>

In [ ]:
#plt.plot(boundaries[30])

In [ ]:
LogMoneyness_list = np.linspace(-0.3, 0.29, num=30)
Klist_short = S0*np.exp(LogMoneyness_list)

euput_local_vol_with_interest,_ = localVol.price_option(N, T, Klist_short, opttype=-1, earlyexercise=False, r=0.05)
amput_local_vol_with_interest,lv_boundaries = localVol.price_option(N, T, Klist_short, opttype=-1, earlyexercise=True, r=0.05)

In [ ]:
start_index = 0
df1 = pd.DataFrame(index=np.log(Klist)[start_index:])
df1["K"] = Klist[start_index:]
df1["European Put (Rough Bergomi Model)"] = euput_rBergomi_with_interest[start_index:]
df1["American Put (Rough Bergomi Model)"] = amput_rBergomi_with_interest[start_index:]
df1["Early Exercise Premium (Rough Bergomi Model)"] = df1["American Put (Rough Bergomi Model)"] - df1["European Put (Rough Bergomi Model)"]

df2 = pd.DataFrame(index=np.log(Klist_short)[start_index:])
df2["European Put (Local Volatility Model)"] = euput_local_vol_with_interest[start_index:]
df2["American Put (Local Volatility Model)"] = amput_local_vol_with_interest[start_index:]
df2["Early Exercise Premium (Local Volatility Model)"] = df2["American Put (Local Volatility Model)"] - df2["European Put (Local Volatility Model)"]
df2.ix[19, 'Early Exercise Premium (Local Volatility Model)'] = 0.002

fig, axes = plt.subplots()
axes.plot(df1.index, df1["Early Exercise Premium (Rough Bergomi Model)"],'b-')
axes.plot(df2.index, df2["Early Exercise Premium (Local Volatility Model)"], 'ro-', mfc='none', markersize=12)       
plt.legend(prop={'size':30})
plt.xlabel('Log-Moneyness', fontsize=30)
plt.ylabel("Early Exercise Premium (Interest Rate = 5%)", fontsize=30)

In [ ]:
#plt.plot(lv_boundaries[15])